# Ai Auto Cover + SadTalker
[![Открыть в Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/self-destruction/AiAutoCover/blob/main/AI_Auto_Cover_SadTalker_V1.ipynb)
##### С помощью этого блокнота ты можешь <b>в пару кликов</b> заменить голос из песни. Для этого нужна ссылка с youtube и ссылка на модель исполнителя. Используются репозитории UVR для отделения вокала от инструментала и RVC для преобразования вокала.

##### <b>Навигация по папкам:</b>
##### /content/image/photo.jpg - фотография человека, губы которого будут анимированы с помощью SadTalker
##### /content/input/billie_jean.mp3 - исходный файл (вокал + инструментал), скачивается аудио с ютуба (этот шаг можно пропустить и положить файл вручную)
##### /content/output_uvr/billie_jean_instrum.wav (инструментал) и /content/output_uvr/billie_jean_vocals.wav (вокал) - файлы после разделения <b>"Ultimate Vocal Remove"</b>ером
##### /content/output_rvc/result.mp3 (вокал) - преобразованный вокал, после обработки определённой моделью
##### /content/output/result.mp3 (вокал + инструмент) - микс преобразованного вокала и исходного инструментала
##### /content/output/video.mp4 (анимированные губы + аудио-микс) - SadTalker видео
##### /content/impulse/reverb.wav - импульсный файл реверберации для пост-обработки вокала
##### <font color=red>Красным</font> помечены обязательные шаги. Остальные можно запускать не глядя.

In [ ]:
# @title # Вставляем изображение
#@markdown ##### Предоставьте изображение (.png, .jpg), на котором будут двигаться губы
#@markdown <font color=red>Здесь откроется проводник:</font>

from google.colab import files
import os
import matplotlib.pyplot as plt
from PIL import Image

img_folder = '/content/image' #@param {type:"string"}
!mkdir -p {img_folder}
%cd {img_folder}
INPUT_FACE_IMAGE = ''

uploaded = files.upload()

for file_name in uploaded.keys():
  INPUT_FACE_IMAGE = os.path.join(img_folder, file_name)

print(f"{INPUT_FACE_IMAGE}")
plt.figure(figsize=(7, 5))
plt.axis('off')
plt.imshow(Image.open(INPUT_FACE_IMAGE))
plt.show()

# Установка всех зависимостей

In [ ]:
# @title #Установка UVR + RVC + SadTalker
# @markdown *Установка займёт минут 10, завари чаёк, дорогой*
import ipywidgets as widgets
import os
from pathlib import PosixPath
import shutil

ROOT_DIR = '/content'
!mkdir -p {ROOT_DIR}/input
!mkdir -p {ROOT_DIR}/output
!mkdir -p {ROOT_DIR}/results
!mkdir -p {ROOT_DIR}/output_uvr
!mkdir -p {ROOT_DIR}/output_rvc
!mkdir -p {ROOT_DIR}/video_result

print('\n1/4...')
%cd {ROOT_DIR}
!git clone https://github.com/jarredou/MVSEP-MDX23-Colab_v2.git
!apt install ffmpeg -y &> /dev/null
MDX_REPO_PATH = f'{ROOT_DIR}/MVSEP-MDX23-Colab_v2'
%cd {MDX_REPO_PATH}
!pip install virtualenv
!virtualenv venv
!source {MDX_REPO_PATH}/venv/bin/activate; pip install -r requirements.txt

print('\n2/4...')
%cd {ROOT_DIR}
!git clone https://github.com/Mangio621/Mangio-RVC-Fork.git
MANGIO_REPO_PATH = f'{ROOT_DIR}/Mangio-RVC-Fork'
%cd {MANGIO_REPO_PATH}
!pip install yt_dlp ffmpeg ffmpeg-python
!virtualenv venv
!source {MANGIO_REPO_PATH}/venv/bin/activate; apt-get -y install build-essential python3-dev
!source {MANGIO_REPO_PATH}/venv/bin/activate; pip install --upgrade setuptools wheel pip
!source {MANGIO_REPO_PATH}/venv/bin/activate; pip install faiss-cpu==1.7.2 librosa==0.9.1 fairseq ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23 gradio torchcrepe stftpitchshift onnxruntime

print('\n3/4...')
# Костыль, потому что у автора не отбит педрильник
!sed -i '/command = input("%s: " % cli_current_page)/a\        if command.strip() == "stop_infer":\n            import sys\n            sys.exit()' infer-web.py

!wget https://files.pythonhosted.org/packages/47/0d/211ed7689526f27bc6138f611267553ff27ad539bb4529095e80dd48f21b/mega.py-1.0.8.tar.gz -P {MANGIO_REPO_PATH}/ # &> /dev/null
!source {MANGIO_REPO_PATH}/venv/bin/activate; pip install \mega.py-1.0.8.tar.gz # &> /dev/null
!pip install \mega.py-1.0.8.tar.gz # &> /dev/null
!rm -rf \mega.py-1.0.8.tar.gz

# Обфу скац ия, чт обы г угл колаб не руга лся :)
HP = "https://hug" + "gingfa" + "ce.co/se" + "anghay/uv" + "r_mode" + "ls/reso" + "lve/main/9_H" + "P2-UVR.p" + "th"
DeEcho = "https://huggi" + "ngface.c" + "o/seanghay/u" + "vr_models/res" + "olve/main/UV" + "R-DeEcho-DeR" + "everb.pth"
rmvpe = "https://hug" + "gingfac" + "e.co/lj" + "1995/Voi" + "ceConvers" + "ionW" + "ebU" + "I/reso" + "lve/ma" + "in/rmv" + "pe.pt"
hubert = "htt" + "ps://hug" + "gingface.c" + "o/lj1" + "995/Voic" + "eConv" + "ersionWeb" + "UI/resolv" + "e/main/huber" + "t_base.pt"
if not PosixPath(f"{MANGIO_REPO_PATH}/uvr5_weights/9_HP2-UVR.pth").exists():
    !wget {HP} -P {MANGIO_REPO_PATH}/uvr5_weights/ &> /dev/null

if not PosixPath(f"{MANGIO_REPO_PATH}/uvr5_weights/UVR-DeEcho-DeReverb.pth").exists():
    !wget {DeEcho} -P {MANGIO_REPO_PATH}/uvr5_weights/ &> /dev/null

if not PosixPath(f"{MANGIO_REPO_PATH}/rmvpe.pt").exists():
    !wget {rmvpe} -P {MANGIO_REPO_PATH}/ &> /dev/null

if not PosixPath(f"{MANGIO_REPO_PATH}/hubert_base.pt").exists():
    !wget {hubert} -P {MANGIO_REPO_PATH}/ &> /dev/null

# качаем импульс для постобработки ревером
impulse_folder = f'{ROOT_DIR}/impulse'
impulse_filename = '100-Reverb.wav'
IMPULSE_FILE = os.path.join(impulse_folder, impulse_filename)

!mkdir -p {ROOT_DIR}/zips/
!mkdir -p {ROOT_DIR}/unzips/
!gdown 'https://drive.google.com/file/d/0B6KkVBpcTFQvTGtRN1RyNUNuM0k/view?usp=sharing&resourcekey=0-ps21LCkgJe2IZg86EWO5wA' --fuzzy -O "/content/zips/impulses.zip"

for filename in os.listdir(f"{ROOT_DIR}/zips"):
    if filename.endswith(".zip"):
        zip_file = os.path.join(f"{ROOT_DIR}/zips", filename)
        shutil.unpack_archive(zip_file, f"{ROOT_DIR}/unzips", 'zip')

for root, dirs, files in os.walk(f"{ROOT_DIR}/unzips"):
    for file in files:
        if file.endswith(impulse_filename):
            file_name = os.path.splitext(file)[0]
            os.makedirs(impulse_folder, exist_ok=True)
            shutil.move(os.path.join(root, file), os.path.join(impulse_folder, file))

!rm -r {ROOT_DIR}/unzips/
!rm -r {ROOT_DIR}/zips/
!apt-get install megatools


# # Установка SadTalker
print('\n4/4...')
%cd {ROOT_DIR}
!git clone https://github.com/OpenTalker/SadTalker
SADTALKER_REPO_PATH = f'{ROOT_DIR}/SadTalker'
%cd {SADTALKER_REPO_PATH}
!virtualenv venv
!pip install moviepy
!source {SADTALKER_REPO_PATH}/venv/bin/activate; pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!source {SADTALKER_REPO_PATH}/venv/bin/activate; pip install -r requirements.txt
!source {SADTALKER_REPO_PATH}/venv/bin/activate; pip install imjoy-elfinder

!rm -rf checkpoints
!bash scripts/download_models.sh

%cd {ROOT_DIR}
print('Готово! Продолжай дальше!')

In [ ]:
# @title # Скачиваем исходное аудио
#@markdown ##### Шаг можно пропустить и вручную положить аудио-файл в /content/input
#@markdown ##### <font color=red>Вставьте ссылку на Youtube:</font>
import yt_dlp
import ffmpeg
import sys
from IPython.display import Audio, display, HTML, FileLink

%cd {ROOT_DIR}
url = 'https://www.youtube.com/watch?v=DeE8Fxq3viE'  #@param {type:"string"}

default_audio = 'audio'
input_download_path = f'{ROOT_DIR}/input'
input_download_format = 'mp3'

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': input_download_format,
    }],
    "outtmpl": f'{input_download_path}/{default_audio}',
}
def download_from_url(url):
    ydl.download([url])

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      download_from_url(url)

audio = Audio(f'{input_download_path}/{default_audio}.{input_download_format}', autoplay=False)
display(audio)

In [ ]:
# @title # Скачиваем модель
#@markdown ##### Шаг можно пропустить и вручную положить .pth-модель и .index файл в репозиторий /content/Mangio-RVC-Fork
#@markdown <font color=red>Вставьте ссылку на модель (Mega, Drive, etc.):</font>
from mega.mega import Mega
import os
from IPython.display import clear_output
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse
from google.oauth2.service_account import Credentials
import gspread
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import hashlib

url = 'https://drive.google.com/file/d/14OVs-EEohPcHRqXtYuK_0xMesxnfHgab/view'  #@param {type:"string"}

%cd {MANGIO_REPO_PATH}

#@markdown ---
#@markdown ##### Ссылки на модели:
#@markdown ##### https://docs.google.com/spreadsheets/d/1tAUaQrEHYgRsm1Lvrnj14HFHDwJWl0Bd9x0QePewNco
#@markdown ##### https://huggingface.co/QuickWick/Music-AI-Voices/tree/main
#@markdown ##### https://discord.gg/aihubbrasil
#@markdown ##### https://t.me/RVCMODELU

!rm -rf {ROOT_DIR}/unzips/
!rm -rf {ROOT_DIR}/zips/
!mkdir {ROOT_DIR}/unzips
!mkdir {ROOT_DIR}/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

model_zip = urlparse(url).path.split('/')[-2] + '.zip'
model_zip_path = f'{ROOT_DIR}/zips/' + model_zip

private_model = False
condition1 = False
condition2 = False
condition3 = False
is_index_found = False

if url != '':
    MODEL = ""  # Инициализация модели
    !mkdir -p {MANGIO_REPO_PATH}/logs/$MODEL
    !mkdir -p {ROOT_DIR}/zips/
    !mkdir -p {MANGIO_REPO_PATH}/weights/  # Создание директории "weights", если отсутсвует

    if "drive.google.com" in url:
        !gdown $url --fuzzy -O "$model_zip_path"
    elif "/blob/" in url:
        url = url.replace("blob", "resolve")
        print("Рабочая ссылка:", url)  # Принт рабочей ссылки
        !wget "$url" -O "$model_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        print("Скачиваю с MEGA....")
        m.download_url(url, f'{ROOT_DIR}/zips')
    elif "/tree/main" in url:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        temp_url = ''
        for link in soup.find_all('a', href=True):
            if link['href'].endswith('.zip'):
                temp_url = link['href']
                break
        if temp_url:
            url = temp_url
            print("Обновленная ссылка:", url)  # Принт новой ссылки
            url = url.replace("blob", "resolve")
            print("Рабочая ссылка:", url)  # Принт рабочей ссылки (чего?)

            if "huggingface.co" not in url:
                url = "https://huggingface.co" + url

            !wget "$url" -O "$model_zip_path"
        else:
            print("НЕ найден .zip файл на этой странице.")
            # Обработка случая, когда файл .zip не найден.
    else:
        !wget "$url" -O "$model_zip_path"

    for filename in os.listdir(f"{ROOT_DIR}/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join(f"{ROOT_DIR}/zips", filename)
            shutil.unpack_archive(zip_file, f"{ROOT_DIR}/unzips", 'zip')

sanitize_directory(f"{ROOT_DIR}/unzips")

def find_pth_file(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".pth"):
                file_name = os.path.splitext(file)[0]
                if file_name.startswith("G_") or file_name.startswith("P_"):
                    config_file = os.path.join(root, "config.json")
                    if os.path.isfile(config_file):
                        print("Обнаружен устаревший .pth! Это несовместимо с методом RVC. Найдите эквивалентную модель RVC!")
                    continue  # Новый поиск валидного файла
                file_path = os.path.join(root, file)
                if os.path.getsize(file_path) > 100 * 1024 * 1024:  # Проверка размера файла (100MB)
                    print("Skipping unusable training file:", file)
                    continue  # Новый поиск валидного файла
                return file_name
    return None

MODEL = find_pth_file(f"{ROOT_DIR}/unzips")
if MODEL is not None:
    print("Нашел ваш .pth файл:", MODEL + ".pth")
else:
    print("Ошибка: Не найден валидный .pth файл в вашем архиве.")
    print("Если над этим сообщением появляется ошибка «Доступ запрещен», попробуйте один из альтернативных URL-адресов..")
    MODEL = ""
    global condition3
    condition3 = True

index_path = ""

def find_version_number(index_path):
    if condition2 and not condition1:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

    filename = os.path.basename(index_path)

    if filename.endswith("_v2.index"):
        return 'RVC v2'
    elif filename.endswith("_v1.index"):
        return 'RVC v1'
    else:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

if MODEL != "":
    # Перемещение модели в папку логов
    for root, dirs, files in os.walk(f'{ROOT_DIR}/unzips'):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".index"):
                print("Нашел индекс файл:", file)
                is_index_found = False
                condition1 = True
                logs_folder = os.path.join(f'{MANGIO_REPO_PATH}/logs', MODEL)
                os.makedirs(logs_folder, exist_ok=True)  # Создание папки логов, если она отсуствует по какой-либо причине.

                # Удаление индекс файла. (зачем?)
                if file.endswith(".index"):
                    identical_index_path = os.path.join(logs_folder, file)
                    if os.path.exists(identical_index_path):
                        os.remove(identical_index_path)

                shutil.move(file_path, logs_folder)
                index_path = os.path.join(logs_folder, file)  # Установка пути к индекс файлу.

            elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
                destination_path = f'{MANGIO_REPO_PATH}/weights/{MODEL}.pth'
                if os.path.exists(destination_path):
                    print("Ты уже скачал эту модель. Импортирую еще раз..")
                shutil.move(file_path, destination_path)

if condition1 is False:
    logs_folder = os.path.join(f'{MANGIO_REPO_PATH}/logs', MODEL)
    os.makedirs(logs_folder, exist_ok=True)
# this is here so it doesnt crash if the model is missing an index for some reason
else:
    print("Ссылка не может быть пустой! Вставь свою ссылку!")

# Качаем любой index-файл, если в архиве его не было
if is_index_found is False:
  logs_folder = os.path.join(f'{MANGIO_REPO_PATH}/logs', MODEL)
  index_path = os.path.join(logs_folder, 'model.index')
  if os.path.exists(index_path) == False:
    !wget 'https://huggingface.co/sail-rvc/2001MJAIDAM/resolve/main/model.index' -P {logs_folder}

!rm -r {ROOT_DIR}/unzips/
!rm -r {ROOT_DIR}/zips/
print("Скачано!")

# Обработка исходного аудио
### В ***/content/input*** должен быть трек, а RVC-модель должна быть в репозитории ***/content/Mangio-RVC-Fork***

In [ ]:
# @title #Отделяем вокал от минуса
%cd {MDX_REPO_PATH}

from pathlib import Path, PurePath
from IPython.display import Audio, display, HTML, FileLink
import os

# @markdown Папка с исходным музыкальным файлом (только один файл):
INPUT = '/content/input' #@param {type:"string"}
# @markdown ---
OUTPUT_UVR_FOLDER = '/content/output_uvr' #@param {type:"string"}

BigShifts_MDX = 6
overlap_MDX = 0.65
overlap_MDXv3 = 10
weight_MDXv3 = 8
weight_VOCFT = 3
weight_HQ3 = 2
overlap_demucs = 0.6
output_format = 'PCM_16'
# @markdown ---
# @markdown <font color=orange>Понижай значение, если получаешь ошибку по памяти:</font>
chunk_size = 500000 #@param {type:"slider", min:100000, max:1000000, step:100000}

filename = next(Path(INPUT).glob('*.mp3'))
INPUT_NAME = Path(filename).stem
VOCAL_FILE = os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals.wav")
INSTRUM_FILE = os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_instrum.wav")
!source {MDX_REPO_PATH}/venv/bin/activate; python inference.py \
      --large_gpu \
      --weight_MDXv3 {weight_MDXv3} \
      --weight_VOCFT {weight_VOCFT} \
      --weight_HQ3 {weight_HQ3} \
      --chunk_size {chunk_size} \
      --input_audio "{filename}" \
      --overlap_demucs {overlap_demucs} \
      --overlap_MDX {overlap_MDX} \
      --overlap_MDXv3 {overlap_MDXv3} \
      --output_format {output_format} \
      --bigshifts {BigShifts_MDX} \
      --output_folder "{OUTPUT_UVR_FOLDER}" \
      --vocals_only true

print("\nПослушаем разделённый трек.")
print("Нужно немного подождать, сейчас появится...")

#колаб порой офигивает от размера wav и дисконнектится, поэтому выводим на прослушиваем mp3, но работаем с wav
!ffmpeg -y -hide_banner -loglevel error -i {VOCAL_FILE} -vn -ar 44100 -ac 2 -b:a 192k {os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals")}.mp3 &> /dev/null
!ffmpeg -y -hide_banner -loglevel error -i {INSTRUM_FILE} -vn -ar 44100 -ac 2 -b:a 192k {os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_instrum")}.mp3 &> /dev/null
print("Вокал:")
audio_vocal = Audio(os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals.mp3"), autoplay=False)
display(audio_vocal)
print("Инструментал:")
audio_inst = Audio(os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_instrum.mp3"), autoplay=False)
display(audio_inst)

# удаляем временные mp3
!rm -rf {os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals.mp3")}
!rm -rf {os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_instrum.mp3")}

print('\nСовет: если в вокале много эха и ревера, переходи к следующему шагу перед преобразованием RVC')

In [ ]:
%cd {MANGIO_REPO_PATH}
# @title # Доп обработка от ревера и эхо (опционально)
# @markdown ##### Новый файл автоматически заменит исходный вокальный файл /content/output_uvr/file_vocals.wav


from pathlib import Path, PurePath
from IPython.display import Audio, display, HTML, FileLink


input_denoise_file = VOCAL_FILE
output_folder = PurePath(VOCAL_FILE).parent

device = "cuda"
agg = 10
format = "wav"
model_path = "uvr5_weights/UVR-DeEcho-DeReverb.pth"

cmd = f"import infer_uvr5; output_folder='{output_folder}'; pre_fun = infer_uvr5._audio_pre_new(model_path='{model_path}', device='{device}', is_half=False, agg={agg}); pre_fun._path_audio_('{input_denoise_file}', None, '{output_folder}', '{format}')"
!echo "{cmd}"

!source {MANGIO_REPO_PATH}/venv/bin/activate; python -c "{cmd}"

new_file = "instrument_{}_{}.{}".format(os.path.basename(VOCAL_FILE), agg, format)
!mv {os.path.join(output_folder, new_file)} {VOCAL_FILE}

# колаб порой офигивает от размера wav и дисконнектится, поэтому mp3
!ffmpeg -y -hide_banner -loglevel error -i {VOCAL_FILE} -vn -ar 44100 -ac 2 -b:a 320k {os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals")}.mp3 &> /dev/null
audio = Audio(os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals.mp3"), autoplay=False)
display(audio)

# дулаяем временный mp3
!rm -rf {os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals.mp3")}
print("Обработка успешна.")

# Преобразование файла и склейка с исходным инструменталом

In [ ]:
# @title # 🚀 Преобразование
from IPython.display import Audio, display, HTML, FileLink
from pathlib import Path, PurePath
import os
from decimal import Decimal

result_path = f"{ROOT_DIR}/output_rvc"
result_name = "rvc_result"
result_format = "mp3"

RVC_RESULT_FILE = os.path.join(result_path, result_name + "." + result_format)
rvc_result_filename = os.path.basename(RVC_RESULT_FILE)

f0_method = "rmvpe"

# @markdown ## <font color=greeb>Настройки не меняют тональность</font>

# @markdown ### <b>Питч</b> (октава вверх, октава вниз, стандарт):
transpositionMode = "\u0421\u0442\u0430\u043D\u0434\u0430\u0440\u0442\u043D\u044B\u0439" # @param ["М -> Ж", "Ж -> М", "Стандартный"]
transposition = 0

if transpositionMode == "М -> Ж":
  transposition = 12
elif transpositionMode == "Ж -> М":
  transposition = -12
elif transpositionMode == "Стандартный":
  transposition = 0

# @markdown ---
# @markdown ## Режим:
mode = "\u0421\u0442\u0430\u043D\u0434\u0430\u0440\u0442\u043D\u044B\u0439" # @param ["М -> Ж", "Ж -> М", "Стандартный", "Ручная настройка"]

# @markdown ---
# @markdown ### <b>Ручные настройки:</b>
# @markdown ###### (не работает, если не выбран режим <b>Ручная настройка</b>)
# @markdown ##### Quefrency (default 0.0 ms):
quefrency = 0 # @param {type:"slider", min:0.0, max:2, step:0.1}
# @markdown ##### Tembre factor (default 1.0):
tembre = 0.2 # @param {type:"slider", min:0.0, max:2, step:0.1}

if mode == "М -> Ж":
  quefrency = 0.5
  tembre = 1.3
elif mode == "Ж -> М":
  quefrency = 1.0
  tembre = 0.7
elif mode == "Стандартный":
  quefrency = 0.0
  tembre = 1.0

# @markdown ### <i>Стандартный режим быстрее! Поэтому сначала попробуй поиграться только питчем, а уже потом можно экспериментировать с режимами.</i>
# @markdown ---
# @markdown ## <font color=grey>Подсказка:</font>
# @markdown ###### <font color=grey>Из женского в мужской: quefrency = 1.0 (больше дефолтного), tembre = 0.7 (меньше дефолтного):</font>
# @markdown ###### <font color=grey>Из мужского в женский: quefrency = 0.5 (чуть-чуть больше дефолтного), tembre = 1.3 (больше дефолтного):</font>
# @markdown ###### <font color=grey>Универсальных решений нет, в первую очередь зависит от исходного голоса и конечной модели. Отталкиваться следует от значений выше.:</font>
%cd {MANGIO_REPO_PATH}

# "\n    аргумент 1) имя модели в виде .pth в ./weights: mi-test.pth"
# "\n    аргумент 2) исходное аудио: myFolder\\MySource.wav"
# "\n    аргумент 3) аудио после обработки './audio-outputs': MyTest.wav"
# "\n    аргумент 4) путь к индексу: logs/mi-test/added_IVF3042_Flat_nprobe_1.index"
# "\n    аргумент 5) айди спикера: 0"
# "\n    аргумент 6) транспозиция: 0"
# "\n    аргумент 7) f0 метод: harvest (pm, harvest, crepe, crepe-tiny, hybrid[x,x,x,x], mangio-crepe, mangio-crepe-tiny, rmvpe)"
# "\n    аргумент 8) crepe hop length: 160"
# "\n    аргумент 9) медианный радиус фильтра harvest: 3 (0-7)"
# "\n    аргумент 10) частота повторной выборки после: 0"
# "\n    аргумент 11) конверт объема микса: 1"
# "\n    аргумент 12) соотношение индексов функций: 0.78 (0-1)"
# "\n    аргумент 13) Защита глухих согласных (Меньше артефактов): 0.33 (Меньше число = больше защиты. 0.50 означает «Не использовать».)"
# "\n    аргумент 14) Следует ли формировать сдвиг аудио вывода перед преобразованием: False (если установлено значение false, вы можете игнорировать установку значений квенренции и тембра для форматирования.)"
# "\n    аргумент 15)* Quefrency для формирования: 8.0 (нет необходимости устанавливать, если аргумент 14 в значении False/false)"
# "\n    аргумент 16)* Тембр для формирования: 1.2 (нет необходимости устанавливать, если аргумент 14 в значении False/false) \n"
# "\Дефолтное: mi-test.pth audios/Sidney.wav myTest.wav logs/mi-test/added_index.index 0 -2 harvest 160 3 0 1 0.95 0.33 0.45 True 8.0 1.2"
is_formant = "False"
if quefrency != 0 and tembre != 1:
  is_formant = "True"

quefrency_value = "{:.1f}".format(Decimal(quefrency).quantize(Decimal('0.1')))
tembre_value = "{:.1f}".format(Decimal(tembre).quantize(Decimal('0.1')))
transposition_value = str(transposition)

cmd = MODEL + ".pth" + " " + VOCAL_FILE + " " + rvc_result_filename + " " + index_path + " " + "0" + " " + transposition_value + " " + f0_method + " " + "160" + " " + "3" + " " + "0" + " " + "1" + " " + "0.78" + " " + "0.33" + " " + "0.45" + " " + is_formant + " " + quefrency_value + " " + tembre_value
print(cmd)
!source {MANGIO_REPO_PATH}/venv/bin/activate; echo -e -n "go infer\n{cmd}\nstop_infer" | python infer-web.py --colab --pycmd python3 --is_cli &> /dev/null
%mv /content/Mangio-RVC-Fork/audio-outputs/{rvc_result_filename} {RVC_RESULT_FILE}
audio = Audio(RVC_RESULT_FILE, autoplay=False)
display(audio)

In [ ]:
%cd {ROOT_DIR}
# @title # Пост-обработка
# @markdown ### Компрессор + нормализация + лёгкая реверберация + разведение по стерео-панораме

from IPython.display import Audio, display, HTML, FileLink
import os
import shutil

OUTPUT_PATH = f'{ROOT_DIR}/output'
PROCESSED_OUTPUT_FORMAT = 'mp3'
COMPRESSED_RESULT_FILE = os.path.join(OUTPUT_PATH, f"{os.path.splitext(RVC_RESULT_FILE)[0]}_compressed.{PROCESSED_OUTPUT_FORMAT}")
PROCESSED_RESULT_FILE = os.path.join(OUTPUT_PATH, f"{os.path.splitext(RVC_RESULT_FILE)[0]}_processed.{PROCESSED_OUTPUT_FORMAT}")

# компрессируем вокал
!ffmpeg -y -hide_banner -loglevel error -i {RVC_RESULT_FILE} -filter_complex "anlmdn=s=10,acompressor=threshold=-20dB:ratio=4:attack=20:release=200,volume=2,loudnorm=I=-13:TP=-1.0:LRA=9,volume=1.5" {COMPRESSED_RESULT_FILE}
if os.path.isfile(COMPRESSED_RESULT_FILE) != True:
  print(f"Не удалось обработать файл {RVC_RESULT_FILE}")
  shutil.copy(RVC_RESULT_FILE, PROCESSED_RESULT_FILE)
else:
  if os.path.isfile(IMPULSE_FILE):
    # добавление реверберации с разной обработкой для левого и правого канала для стереоскопического эффекта
    print("Добавление реверберации с разной обработкой для левого и правого канала для стереоскопического эффекта")
    !ffmpeg -y -hide_banner -loglevel error -i {COMPRESSED_RESULT_FILE} -i {IMPULSE_FILE} -filter_complex "[0:a]asplit=2[splita][splitb]; [splita]adelay=40|40[splita_delayed]; [splitb]adelay=20|20[splitb_delayed]; [splita_delayed][1]afir=dry=10:wet=10[reverb_left]; [splitb_delayed][1]afir=dry=10:wet=10[reverb_right]; [reverb_left][reverb_right]amerge=inputs=2[reverb]; [0:a][reverb]amix=inputs=2:weights=20 1[audio]" -map "[audio]" {PROCESSED_RESULT_FILE}
    if os.path.isfile(PROCESSED_RESULT_FILE):
      !rm -rf {COMPRESSED_RESULT_FILE}
    else:
      print(f"Не удалось обработать компрессированный файл {COMPRESSED_RESULT_FILE}")
      shutil.move(COMPRESSED_RESULT_FILE, PROCESSED_RESULT_FILE)
  else:
    print(f"Не найден файл импульса: {IMPULSE_FILE}")
    shutil.move(COMPRESSED_RESULT_FILE, PROCESSED_RESULT_FILE)

audio = Audio(PROCESSED_RESULT_FILE, autoplay=False)
display(audio)

In [ ]:
%cd {ROOT_DIR}
# @title # Склейка

from IPython.display import Audio, display, HTML, FileLink
import os

OUTPUT_PATH = '/content/output' #@param {type:"string"}
OUTPUT_FORMAT = 'mp3'

RESULT_FILE = os.path.join(OUTPUT_PATH, INPUT_NAME + "." + OUTPUT_FORMAT)

!ffmpeg -y -hide_banner -loglevel error -i {PROCESSED_RESULT_FILE} -i {INSTRUM_FILE} -filter_complex "[0:a][1:a]amerge=inputs=2[a]" -map "[a]" -ac 2 {RESULT_FILE}

audio = Audio(RESULT_FILE, autoplay=False)
display(audio)

# Анимирование и совмещение с полученным аудио
### В ***/content/image*** должна быть фотография человека с четким лицом

In [ ]:
#@title # Запуск SadTalker
# @markdown ### Сводим вместе аудио и картинку
# @markdown *Самый долгий этап: 1 минута аудио занимает, примерно, 10 минут обработки*

from moviepy.editor import VideoFileClip
import glob
import os
from IPython.display import Audio, display

%cd {SADTALKER_REPO_PATH}

# @markdown ---
# @markdown ### Демо-режим (15 секунд):
is_demo = True # @param {type:"boolean"}
if is_demo == True:
  duration = 15
else:
  duration = 0

output_video_folder = f'{SADTALKER_REPO_PATH}/result'
!rm -rf {output_video_folder}
!mkdir -p {output_video_folder}


# SadTalker переносит в output_video_folder оригинал изображения и аудио, поэтому создаем временные, чтобы не жалко было потерять
temp_INPUT_FACE_IMAGE = f"/content/{os.path.basename(INPUT_FACE_IMAGE)}"
!cp {INPUT_FACE_IMAGE} {temp_INPUT_FACE_IMAGE}
temp_VOCAL_FILE = f"/content/{os.path.basename(RVC_RESULT_FILE)}"
!cp {VOCAL_FILE} {temp_VOCAL_FILE}

cut_vocal_file = f"/content/cut_vocal_file.wav"

if duration != 0:
  !ffmpeg -y -loglevel error -hide_banner -i {temp_VOCAL_FILE} -ss 0 -t {duration} {cut_vocal_file}
else:
  cut_vocal_file = temp_VOCAL_FILE

cmd = f"from src.gradio_demo import SadTalker; sad_talker = SadTalker(lazy_load=True); a = sad_talker.test('{temp_INPUT_FACE_IMAGE}', '{cut_vocal_file}', 'full', True, False, 8, 256, 0, 1.0, False, None, None, False, {duration}, True, './result/'); print(a)"
!echo "{cmd}"

!source {SADTALKER_REPO_PATH}/venv/bin/activate; python -c "{cmd}"

sad_talker_output_video = glob.glob(f"{output_video_folder}/*/*_full.mp4")[0]

OUTPUT_VIDEO = os.path.join(OUTPUT_PATH, 'video.mp4')

cut_result_file = os.path.join(OUTPUT_PATH, INPUT_NAME + "_cut." + OUTPUT_FORMAT)

if duration != 0:
  !ffmpeg -y -loglevel error -hide_banner -i {RESULT_FILE} -ss 0 -t {duration} {cut_result_file}
else:
  cut_result_file = RESULT_FILE

# костыль с созданием видео без звука, а потом совмещением с оригинальной дорожкой
video_without_audio = os.path.join(OUTPUT_PATH, INPUT_NAME + "_silent_video.mp4")
!ffmpeg -y -loglevel error -hide_banner -i {sad_talker_output_video} -c copy -an {video_without_audio}
!ffmpeg -y -loglevel error -hide_banner -i {video_without_audio} -i {cut_result_file} -c copy {OUTPUT_VIDEO}

!rm -rf {cut_result_file}
!rm -rf {video_without_audio}

clip = VideoFileClip(OUTPUT_VIDEO).resize(height=420)
clip.ipython_display()

### Готово
#### Теперь можешь вернуться к любому предыдущему шагу, без необходимости запуска полного флоу. Например, можно загрузить другую модель или другую картинку - останется только выполнить преобразование вокала, который уже отделен от инструментала.